**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Jun  6 14:36:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# \content == root folder  
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/drive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive': File exists
/mydrive


**1) Clone the Darknet**



In [0]:
import os
PATH_dir = '/content/gdrive/My Drive/Colab Notebooks/ARpalus-Challenge'
os.chdir( PATH_dir )

if 'darknet' not in os.listdir(PATH_dir):
    !git clone https://github.com/AlexeyAB/darknet darknet

**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/gdrive/My Drive/Colab Notebooks/ARpalus-Challenge/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warni

**3) Configure Darknet network for training YOLO V3**

Based on the following [instructions](https://github.com/AlexeyAB/darknet#how-to-train-to-detect-your-custom-objects) , note the line number to be

In [0]:
# Create identical file with the same content as in yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

# Modifies properties - darknet/cfg/yolov3_training.cfg
# TODO : Dynamic allocations for parameters w.r.t Data 
# ------------------------------------------------- #
# NOTE : my batches are very small
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg   # !sed == replace {text by text}
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
# Given n classes - n x num_classes
!sed -i 's/max_batches = 500200/max_batches = 54000/' cfg/yolov3_training.cfg
# Given steps = [80%, 90%] * max_batches
!sed -i 's/steps=400000,450000/steps=43200,48600/' cfg/yolov3_training.cfg

# ------------- Change num of classes ------------- #
# change line classes=80 to your number of objects in each of 3 [yolo]-layers
!sed -i '610 s@classes=80@classes=27@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=27@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=27@' cfg/yolov3_training.cfg
# ------------- Change num of filters ------------- #
# change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer
!sed -i '603 s@filters=255@filters=96@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=96@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=96@' cfg/yolov3_training.cfg

# (!) VERIFY :: in text file itself (!)

In [0]:
# Create folder on google drive so that we can save there the weights
if 'yolov3' not in os.listdir(PATH_dir):
    !mkdir PATH_dir+'/yolov3'

PATH_yolov3 = PATH_dir + '/yolov3'
# Verify list of classses and their corresponding labels 

!echo -e 'classes= 27\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = backup' > data/obj.data
# !echo -e 'classes= 27\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/gdrive/My Drive/Colab Notebooks/ARpalus-Challenge/yolov3' > data/obj.data
# !echo -e '---list of classes---' > data/obj.names

In [0]:
# Download weights darknet model 53
PATH_darknet = PATH_dir + '/darknet'
NAME_MODEL = 'darknet53.conv.74'
if NAME_MODEL not in os.listdir(PATH_darknet):
    !wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [8]:
# UNZIP dataset from "yolov3" >> "data/obj"
# !pwd
# PATH_zip = '/content/gdrive/My Drive/Colab Notebooks/ARpalus-Challenge/Uploaded/AR_test.zip' # <==> 
PATH_zip = PATH_dir + '/Uploaded/AR_train.zip' # <==> path not recognize (!) check why
!unzip AR_train.zip -d data/obj  # put manually in directory

Archive:  AR_train.zip
replace data/obj/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
import glob, random

labels_list = glob.glob("data/obj/*.txt")    # extract labels ended with '.txt'
images_list_jpg = glob.glob("data/obj/*.jpg") # extract images ended with '.jpg / .jpeg / .JPG'
images_list_jpeg = glob.glob("data/obj/*.jpeg")
images_list_JPG = glob.glob("data/obj/*.JPG")

train_list = images_list_jpg + images_list_jpeg + images_list_JPG

# print('Images train list ( .jpg ) : \n' ); random.sample( images_list, 15 )
# print('Labels List ( .txt ) : \n' ); random.sample( labels_list, 15 ) 

In [0]:
# Create training file
file = open("data/train.txt", "w") 
file.write("\n".join(train_list)) 
file.close() 

# Create testing file
file = open("data/test.txt", "w") 
test_portion = round(0.15*len(train_list))
file.write("\n".join( random.choices(train_list, k=test_portion) )) 
file.close() 


**6) Train the Model** ( from scratch / *Transfer learning* )

In [12]:
# ------------------- Training from scratch ----------------- #
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

# -------- Transfer Learning (resume from last save) -------- #
!./darknet detector train data/obj.data cfg/yolov3_training.cfg yolov3_training_last.weights darknet53.conv.74 -dont_show

v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.735952, GIOU: 0.731301), Class: 0.992222, Obj: 0.945168, No Obj: 0.005027, .5R: 1.000000, .75R: 0.250000, count: 8, class_loss = 0.128464, iou_loss = 0.493581, total_loss = 0.622045 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.670860, GIOU: 0.656665), Class: 0.936432, Obj: 0.757915, No Obj: 0.002056, .5R: 0.812500, .75R: 0.375000, count: 16, class_loss = 0.889814, iou_loss = 1.724444, total_loss = 2.614259 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.642113, GIOU: 0.624616), Class: 0.775955, Obj: 0.673373, No Obj: 0.000459, .5R: 0.857143, .75R: 0.214286, count: 14, class_loss = 1.896409, iou_loss = 1.741793, total_loss = 3.638202 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.699178, GIOU: 0.692778), Class: 0.985601, Obj: 0.801907, No Obj: 0.003751, .5R: 1.000000, .75R: 0.333333, count: 6, class_loss = 0.267312, iou_loss = 0.511276, tota

---
#### **TODO** : Implement a video fotage for a more robust validation

---
